In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as BS

In [2]:
url="https://cloud.google.com/release-notes/all"
page=rq.get(url).content
soup=BS(page,'html.parser')

In [3]:
services = soup.findAll("li", {"class": "release-note-sitemap-card-item"})

In [4]:
root_url="https://cloud.google.com/"
url_pack=[dict(name=i.find("h3").text,url=root_url+i.find("a")["href"]) for i in services]
source=pd.DataFrame(url_pack)
source

,name,url
0,AutoML Natural Language,https://cloud.google.com//automl-tables/docs/r...
1,AutoML Tables,https://cloud.google.com//natural-language/aut...
2,AutoML Translation,https://cloud.google.com//translate/automl/doc...
3,AutoML Video Intelligence Classification,https://cloud.google.com//video-intelligence/a...
4,AutoML Video Intelligence Object Tracking,https://cloud.google.com//video-intelligence/a...
...,...,...
144,Web Risk API,https://cloud.google.com//web-risk/docs/releas...
145,Cloud Run,https://cloud.google.com//run/docs/release-notes
146,Cloud Functions,https://cloud.google.com//functions/docs/relea...
147,Cloud Storage,https://cloud.google.com//storage/docs/release...


In [5]:
si=source['url'][0]

In [6]:
si="https://cloud.google.com/ai-platform/docs/release-notes"

In [7]:
su=BS(rq.get(si).content,'html.parser')

In [21]:
releases = su.findAll("section",{"class": "releases"})[1]
print(releases.text[:500])


April 29, 2020
AI Platform Prediction

AI Platform Prediction now supports several regional endpoints for online prediction. Regional endpoints provide additional protection against outages in other regions by isolating your model and version resources from other regions. The following regional endpoints are available in beta:

us-central1-ml.googleapis.com
europe-west4-ml.googleapis.com
asia-east1-ml.googleapis.com

You can use these endpoints instead of the global endpoint, ml.googleapis.com,


In [26]:
dates_of_release = releases.findAll("h2" , attrs= {'tabindex': '0'})
dates_of_release[0]['id']

'April_29_2020'

In [29]:
dates_released = []
for date in dates_of_release:
    dates_released.append(date['id'])
dates_released[:5]

['April_29_2020',
 'April_24_2020',
 'April_13_2020',
 'April_09_2020',
 'April_08_2020']

In [30]:
release_descriptions = releases.findAll("strong", class_ = "release-note-product-title")
release_descriptions[0].text

'AI Platform Prediction'

In [32]:
releases_desc = []
for release_description in release_descriptions:
    releases_desc.append(release_description.text)
releases_desc[:5]

['AI Platform Prediction',
 'AI Platform Prediction',
 'AI Platform Prediction',
 'AI Platform Prediction',
 'AI Platform Training']